# Stable Diffusion + Telegram bot
[Github Repo](https://github.com/blackestwhite/tele-sd)

## Donations
Ethereum: `0xE76fc1CE4d3ffFEA12A5686618844408C327357b`

## steps
- create a telegram bot
- set the token on the code
- run all


## How to use
go to your bot on telegram send the Prompts separated by `__`, (two underscores).
before the separator will be the positive prompt, after that will be the negative prompt

## Mount Google Drive

In [ ]:
import os
from google.colab import drive

drivePath = '/content/drive'

if os.path.isdir(drivePath) == False:
    drive.mount(drivePath)

## Install Requirements

In [ ]:
%pip install --quiet nest-asyncio transformers diffusers python-telegram-bot accelerate

## Import and Setup Bot + SD

In [ ]:
from telegram import Update, Bot
from telegram.ext import ApplicationBuilder, CommandHandler, ContextTypes, MessageHandler, filters
import nest_asyncio
from diffusers import StableDiffusionPipeline, EulerDiscreteScheduler
import torch
import asyncio
import random

model_id = "runwayml/stable-diffusion-v1-5"
scheduler = EulerDiscreteScheduler.from_pretrained(model_id, subfolder="scheduler")



if os.path.isdir(f'{drivePath}/MyDrive/sd_1-5'):
    pipe = StableDiffusionPipeline.from_pretrained(f'{drivePath}/MyDrive/sd_1-5', torch_dtype=torch.float16, scheduler=scheduler)
else:
    pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16, scheduler=scheduler)
    pipe.save_pretrained(f'{drivePath}/MyDrive/sd_1-5')


pipe = pipe.to("cuda")
pipe.enable_attention_slicing() # to use less memory
pipe.safety_checker = None # to disable NSFW filter

nest_asyncio.apply()
token = "Your Telegram Bot Token Here"

## Setup Handler for bot

In [ ]:
default_steps = 20

async def generate(update: Update, context: ContextTypes.DEFAULT_TYPE) -> None:
  text = update.message.text
  prompt = ''
  neg = ''
  if '__' in text:
    prompt = text.split("__")[0]
    neg = text.split("__")[1]
  else:
    prompt = text
  seed = random.randint(0, 2147483647)
  image = pipe(
    prompt,
    negative_prompt=neg,
    num_inference_steps=default_steps,
    height=512,
    width=512,
    guidance_scale = 8,
    generator = torch.Generator("cuda").manual_seed(seed)
    ).images[0]
  image.save("./image.png")
  await update.message.reply_photo(photo="./image.png")

app = ApplicationBuilder().token(token).build()

app.add_handler(MessageHandler(filters.TEXT & ~filters.COMMAND, generate))

app.run_polling()